In [1]:
# Set up logging

import logging

logging.basicConfig(
    format="[%(asctime)s %(levelname)s %(threadName)s %(name)s:%(funcName)s:%(lineno)s] %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S%z",
)
logging.getLogger("council").setLevel(logging.INFO)

In [2]:
# Council imports

from council.runners import Budget
from council.contexts import AgentContext, ChatHistory
from council.agents import Agent
from council.chains import Chain
from council.llm.openai_llm_configuration import OpenAILLMConfiguration
from council.llm.openai_llm import OpenAILLM

from skills import SectionWriterSkill, OutlineWriterSkill
from controller import WritingAssistantController
from evaluator import BasicEvaluatorWithSource

In [3]:
# Read environment variables and initialize OpenAI LLM instance

import dotenv
dotenv.load_dotenv()
openai_llm = OpenAILLM(config=OpenAILLMConfiguration.from_env())

In [4]:
# Create Skills

outline_skill = OutlineWriterSkill(openai_llm)
writing_skill = SectionWriterSkill(openai_llm)

In [5]:
# Create Chains

outline_chain = Chain(
    name="Outline Writer",
    description="Write or revise the outline (i.e. section headers) of a research article in markdown format. Only use this chain when the ARTICLE OUTLINE is missing or incomplete.",
    runners=[outline_skill]
)

writer_chain = Chain(
    name="Article Writer",
    description="Write or revise specific section bodies of a research article in markdown format. Use this chain to write the main research article content.",
    runners=[writing_skill]
)

In [6]:
# Create Controller

controller = WritingAssistantController(
    openai_llm,
    top_k_execution_plan=3
)

In [7]:
# Initialize Agent

chat_history = ChatHistory()
run_context = AgentContext(chat_history)
agent = Agent(controller, [outline_chain, writer_chain], BasicEvaluatorWithSource())

In [8]:
# Run Agent

chat_history.add_user_message("What is the financial performance of Microsoft?")
result = agent.execute(run_context, Budget(600))

[2023-07-25 13:51:49-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-25 13:51:49-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="1"
[2023-07-25 13:51:54-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Article Writer" execution_unit="Article Writer: Start with writing the 'Introduction' section, providing a brief overview of the topic and the purpose of the research."
[2023-07-25 13:51:54-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="SectionWriterSkill"
[2023-07-25 13:52:03-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2023-07-25 13:52:03-0400 INFO MainThread council.agents.agent:execute:76] messag

In [ ]:
print(result.messages[-1].message.message)

```markdown
# Microsoft's Financial Performance: An Analysis

## Introduction

In the rapidly evolving world of technology, financial performance is a key indicator of a company's health and future prospects. This article aims to delve into the financial performance of one of the tech giants, Microsoft Corporation. We will analyze various financial metrics and trends to provide a comprehensive understanding of Microsoft's financial standing. This analysis will serve as a valuable resource for investors, financial analysts, and anyone interested in the financial trajectory of Microsoft.

## Company Overview

Microsoft Corporation, founded by Bill Gates and Paul Allen in 1975, is a multinational technology company headquartered in Redmond, Washington. The company's business model is based on developing, licensing, and supporting a wide range of software products, services, and devices. Microsoft's major products and services include the Windows operating systems, the Microsoft Office sui

In [ ]:
chat_history.add_user_message("Can you please add a section about environmental priorities? ")
result = agent.execute(run_context, Budget(600))

[2023-07-24 14:11:25-0400 INFO MainThread council.agents.agent:execute:60] message="agent execution started"
[2023-07-24 14:11:25-0400 INFO MainThread council.agents.agent:execute:62] message="agent iteration started" iteration="6"
[2023-07-24 14:11:29-0400 INFO MainThread council.agents.agent:execute:71] message="chain execution started" chain="Article Writer" execution_unit="Article Writer: Write a new section on "Environmental Priorities" discussing Microsoft's environmental initiatives, their impact on the company's reputation and financial performance, and future plans in this area."
[2023-07-24 14:11:29-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:82] message="skill execution started" skill="SectionWriterSkill"
[2023-07-24 14:12:38-0400 INFO chain_Article Writer_0 council.skills.skill_base:execute_skill:85] message="skill execution ended" skill="SectionWriterSkill" skill_message="I've written or edited the article and placed it in the 'data' field."
[2

In [ ]:
print(result.messages[-1].message.message)

```markdown
# Microsoft's Financial Performance and Environmental Priorities: An Analysis

## Introduction

In the rapidly evolving world of technology, financial performance is a key indicator of a company's health and future prospects. This article aims to delve into the financial performance of one of the tech giants, Microsoft Corporation. We will analyze various financial metrics and trends to provide a comprehensive understanding of Microsoft's financial standing. This analysis will serve as a valuable resource for investors, financial analysts, and anyone interested in the financial trajectory of Microsoft. Additionally, we will also explore Microsoft's environmental priorities and their impact on the company's reputation and financial growth.

## Company Overview

Microsoft Corporation, founded by Bill Gates and Paul Allen in 1975, is a multinational technology company headquartered in Redmond, Washington. The company's business model is based on developing, licensing, and supp